In [159]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import NMF, TruncatedSVD
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import f1_score
import pymorphy2
from gensim.models import Word2Vec
from xgboost import XGBClassifier
from sklearn.preprocessing import normalize
import razdel
import nltk
nltk.download('stopwords')
import pymorphy2
import gensim
from tqdm import tqdm_notebook
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andrewbelyaev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
!pip3 install razdel

In [2]:
train_df = pd.read_csv("train.csv")

train_df.head()

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [3]:
test_df = pd.read_csv("test.csv")

test_df.head()

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [4]:
train_df["target"].value_counts()

False    118594
True      16715
Name: target, dtype: int64

In [5]:
from sklearn.metrics import f1_score

In [6]:
X_train = train_df["title"].values
X_test = test_df["title"].values
y_train = train_df["target"].astype(int).values

# Simple baseline

In [90]:
y_pred = [int("порно" in text) for text in X_train]

In [91]:
f1_score(y_train, y_pred)

0.31242758551206484

### Submit

In [92]:
test_df["target"] = [("порно" in text) for text in X_test]

test_df[["id", "target"]].to_csv("simple_baseline.csv", index=False)

!cat simple_baseline.csv | head

id,target
135309,False
135310,False
135311,False
135312,False
135313,False
135314,False
135315,False
135316,False
135317,False
cat: stdout: Broken pipe


### Не все так однозначно

**не порно**:
- Болезни опорно-двигательной системы и импотенция: взаимосвязь
- Транссексуальные рыбы - National Geographic Россия: красота мира в каждом кадре
- Групповая обзорная экскурсия по Афинам - цена €50
- Больного раком Задорнова затравили в соцсетях.
- Гомосексуалисты на «Первом канале»? Эрнст и Галкин – скрытая гей-пара российского шоу-бизнеса | Заметки о стиле, моде и жизни

**порно**:
- Отборная домашка
- Сюзанна - карьера горничной / Susanna cameriera perversa (с русским переводом) 1995 г., DVDRip

# ML baseline

In [131]:
def text_classifier(vectorizer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("classifier", classifier)]
        )

In [135]:
for clf in [LogisticRegression, LinearSVC, SGDClassifier]:
    for vect in [CountVectorizer, TfidfVectorizer]:
        print(clf)
        print(vect)
        print(cross_val_score(text_classifier(vect(), clf()), X_train, y_train, cv=5, scoring='f1').mean())
        print("\n")

<class 'sklearn.linear_model._logistic.LogisticRegression'>
<class 'sklearn.feature_extraction.text.CountVectorizer'>


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0.943034670734541


<class 'sklearn.linear_model._logistic.LogisticRegression'>
<class 'sklearn.feature_extraction.text.TfidfVectorizer'>
0.9144887808196971


<class 'sklearn.svm._classes.LinearSVC'>
<class 'sklearn.feature_extraction.text.CountVectorizer'>
0.951810068505077


<class 'sklearn.svm._classes.LinearSVC'>
<class 'sklearn.feature_extraction.text.TfidfVectorizer'>
0.9548248687376308


<class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'>
<class 'sklearn.feature_extraction.text.CountVectorizer'>
0.9423192603057672


<class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'>
<class 'sklearn.feature_extraction.text.TfidfVectorizer'>
0.8927790186977494




In [115]:
print('TfIdfVectorizer, LogisticRegression:', round(0.9022925463226912, 3))
print('CountVectorizer, LogisticRegression:', round(0.943034670734541, 3))
print('TfIdfVectorizer, MultinomialGaussian:', round(0.8981449177149763, 3))
print('CountVectorizer, MultinomialGaussian:', round(0.8371649914733172, 3))
print('TfIdfVectorizer, SGDClassifier:', round(0.8823506407703879, 3))
print('CountVectorizer, SGDClassifier:', round(0.9428561918803572, 3))
print('TfIdfVectorizer, LinearSVC:', round(0.8826559337315283, 3))
print('CountVectorizer, LinearSVC:', round(0.951810068505077, 3))

TfIdfVectorizer, LogisticRegression: 0.902
CountVectorizer, LogisticRegression: 0.943
TfIdfVectorizer, MultinomialGaussian: 0.898
CountVectorizer, MultinomialGaussian: 0.837
TfIdfVectorizer, SGDClassifier: 0.882
CountVectorizer, SGDClassifier: 0.943
TfIdfVectorizer, LinearSVC: 0.883
CountVectorizer, LinearSVC: 0.952


### Submit

In [18]:
X_test_vectorized = vectorizer.transform(X_test)
    
test_df["target"] = model.predict(X_test_vectorized).astype(bool)

test_df[["id", "target"]].to_csv("ml_baseline.csv", index=False)

!cat ml_baseline.csv | head

id,target
135309,False
135310,False
135311,False
135312,True
135313,False
135314,False
135315,False
135316,False
135317,False
cat: stdout: Broken pipe


### Improvements

In [184]:
df = pd.read_csv("train.csv")

df.head()

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [185]:
test_df = pd.read_csv("test.csv")

test_df.head()

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [186]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=33) #на этом обучим и провалидируем модель
corpus = df.title.values #на этом обучим w2v
train_df = train_df[train_df.title.apply(lambda x: len(x) > 2)] #удалим аномалии
TOKEN_PATTERN = "[а-яё]+"
stopword_set = set(nltk.corpus.stopwords.words('russian'))

In [187]:
lemmatizer = pymorphy2.MorphAnalyzer()
lemmatizer_cache = {}

def lemmatize(token):
    if lemmatizer.word_is_known(token):
        if token not in lemmatizer_cache:
            lemmatizer_cache[token] = lemmatizer.parse(token)[0].normal_form
        return lemmatizer_cache[token]
    return token

In [188]:
def prepare_sentence_dataset(documents):
    tokenized_sentences = []
    for document in tqdm_notebook(documents):
        for sentence in razdel.sentenize(document):
            lemmatized_tokens = [lemmatize(token) for token in re.findall(TOKEN_PATTERN, sentence.text.lower())]
            tokenized_sentences.append(
                [token for token in lemmatized_tokens if token not in stopword_set]
            )
    return tokenized_sentences

sentence_dataset = prepare_sentence_dataset(corpus)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [189]:
word2vec = gensim.models.Word2Vec(size=100, sg=0, window=5, min_count=5, negative=20)

In [190]:
word2vec.build_vocab(sentence_dataset, progress_per=20_000)

In [191]:
len(word2vec.wv.vocab)

14552

In [192]:
word2vec.train(sentence_dataset, total_examples=word2vec.corpus_count, epochs=30, report_delay=5)

(19748677, 24067650)

In [193]:
def embed_text(text, word2index, word_embeddings):
    res = np.array([
            word_embeddings[word2index[word]] for word in text 
            if word in word2index and word not in stopword_set
        ]).mean(0, keepdims=True)
    if res.size == 1:
        return np.zeros(shape=(1, word_embeddings.shape[1]))
    else:
        return res

In [205]:
lemmatized_docs = [[lemmatize(token) for token in text] for text in tqdm_notebook(train_df.title.values)]

cleared_docs = [[token for token in text if token not in stopword_set] for text in lemmatized_docs]


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [206]:
len(train_df.title.values)

107922

In [207]:
len(cleared_docs)

107922

In [208]:
index2word = np.array(word2vec.wv.index2word)

In [209]:
word2index = {word: i for i, word in enumerate(index2word)}

In [210]:
embeddings = word2vec.wv.vectors

In [215]:
train_title2vec = np.concatenate([embed_text(doc, word2index, embeddings) for doc in cleared_docs])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice.
  """


In [216]:
train_title2vec.shape

(107922, 100)

In [217]:
train_df.shape

(107922, 4)

In [218]:
clf = SGDClassifier(class_weight='balanced')
clf.fit(train_title2vec, train_df.target)

SGDClassifier(class_weight='balanced')

In [219]:
f1_score(train_df.target, clf.predict(train_title2vec))

0.48158621377956684

In [220]:
val_title2vec = np.concatenate([embed_text(doc, word2index, embeddings) for doc in val_df.title])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice.
  """
/Users/andrewbelyaev/Library/Python/3.7/lib/python/site-packages/numpy/core/_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [221]:
f1_score(val_df.target, clf.predict(val_title2vec))

0.37227330088336036

In [222]:
clf = XGBClassifier(n_estimators=100, n_jobs=-1)

In [223]:
clf.fit(train_title2vec, train_df.target)

KeyboardInterrupt: 

In [ ]:
f1_score(train_df.target, clf.predict(train_title2vec))

In [ ]:
val_title2vec = np.concatenate([embed_text(doc, word2index, embeddings) for doc in val_df.title])

In [ ]:
f1_score(val_df.target, clf.predict(val_df.title))